In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from simpy import Environment, Resource
import simpy
from sklearn.metrics import mean_squared_error


In [2]:
class Weather:
    def __init__(self):
       self.daily_sunlight_pattern = {
       6: 200, 7: 400, 8: 600, 9: 700, 10: 800,
      11: 900, 12: 1000, 13: 950, 14: 900, 15: 800,
      16: 700, 17: 500, 18: 300, 19: 100
                                     }
       self.rainy_hours = set(random.sample(list(self.daily_sunlight_pattern.keys()), 3))

    def get_sunlight(self, hour):
        if hour in self.rainy_hours:
            return self.daily_sunlight_pattern.get(hour, 50) * 0.3
        return self.daily_sunlight_pattern.get(hour, 50)


In [3]:
class Battery:
    def __init__(self, capacity):
        self.capacity = capacity
        self.charge_level = capacity * 0.5

    def charge(self, power_input):
        self.charge_level = min(self.capacity, self.charge_level + power_input)

    def discharge(self, power_output):
        self.charge_level = max(0, self.charge_level - power_output)


In [4]:
class SolarPanel:
    def __init__(self, efficiency, area):
        self.base_efficiency = efficiency
        self.area = area

    def generate_power(self, sunlight_intensity):
        efficiency_factor = 1 - min(0.15, (sunlight_intensity - 800) / 5000)
        actual_efficiency = self.base_efficiency * efficiency_factor
        return actual_efficiency * self.area * sunlight_intensity


In [5]:
class Supercapacitor:
    def __init__(self, capacity):
        self.capacity = capacity
        self.charge_level = capacity * 0.5

    def provide_boost(self, boost_power):
        boost_available = min(self.charge_level, boost_power)
        self.charge_level -= boost_available
        return boost_available


In [6]:
class Motor:
    def __init__(self, power_rating, efficiency):
        self.power_rating = power_rating  # in Watts
        self.efficiency = efficiency      # as a fraction (e.g., 0.85)

    def energy_required(self, distance, terrain_factor, kerb_weight=400):
        """
        Calculate energy required (Wh) for a trip, scaled by kerb weight.
        - distance: trip distance in km
        - terrain_factor: multiplier for terrain difficulty
        - kerb_weight: vehicle weight in kg (default 400kg)
        """
        # Scale energy linearly based on kerb weight (using 400kg as the reference)
        weight_factor = kerb_weight / 400
        base_energy = (self.power_rating * distance * terrain_factor) / 1000 / self.efficiency
        return base_energy * weight_factor


In [7]:
import random

In [8]:
class ElectricTukTuk:
    def __init__(self, battery_capacity, motor, kerb_weight, top_speed):
        self.battery = Battery(battery_capacity)
        self.motor = motor
        self.kerb_weight = kerb_weight
        self.top_speed = top_speed
        self.total_energy_consumed = 0
        self.total_distance_covered = 0
        self.terrain_energy_usage = {"Flat": 0, "Hill": 0, "Sandy": 0, "Rough": 0, "Downhill": 0}
        self.hourly_data = []

    def drive(self, distance, terrain, speed=None):
        terrain_factor = {"Flat": 1.0, "Hill": 1.5, "Sandy": 1.8, "Rough": 2.0, "Downhill": 0.7}.get(terrain, 1.0)
        energy_needed = self.motor.energy_required(distance, terrain_factor)
        if self.battery.charge_level >= energy_needed:
            self.battery.discharge(energy_needed)
            self.total_energy_consumed += energy_needed
            self.total_distance_covered += distance
            self.terrain_energy_usage[terrain] += energy_needed
            wh_per_km = self.total_energy_consumed / self.total_distance_covered
            print(f"Energy Efficiency: {wh_per_km:.2f} Wh/km | Terrain: {terrain}")
        else:
            print("Warning: Not enough battery to complete trip!")

        # Optionally trigger regenerative braking if going downhill and speed is given
        if terrain == "Downhill" and speed is not None:
            self.regenerative_braking(speed, terrain)

    def regenerative_braking(self, speed, terrain):
        if terrain == "Downhill":
            motor_power = getattr(self.motor, 'power_rating', 5000)
            recovered_energy = (motor_power * speed * 0.05)
            self.battery.charge(recovered_energy)
            print(f"Regenerative braking recovered {recovered_energy:.2f} Wh")


In [9]:
class SolarTukTuk:
    def __init__(self, battery_capacity, capacitor_capacity, motor, kerb_weight, top_speed, panel_area, panel_efficiency):
        self.battery = Battery(battery_capacity)
        self.supercapacitor = Supercapacitor(capacitor_capacity)
        self.motor = motor  # Instance of Motor class
        self.kerb_weight = kerb_weight
        self.top_speed = top_speed
        self.solar_panel = SolarPanel(panel_efficiency, panel_area)
        self.weather = Weather()
        self.total_energy_consumed = 0
        self.total_distance_covered = 0
        self.terrain_energy_usage = {"Flat": 0, "Hill": 0, "Sandy": 0, "Rough": 0, "Downhill": 0}
        self.hourly_data = []

    def drive(self, distance, terrain, speed=None):
        terrain_factor = {"Flat": 1.0, "Hill": 1.5, "Sandy": 1.8, "Rough": 2.0, "Downhill": 0.7}.get(terrain, 1.0)
        energy_needed = self.motor.energy_required(distance, terrain_factor)
        boost_needed = min(energy_needed * 0.2, self.supercapacitor.charge_level)

        self.supercapacitor.provide_boost(boost_needed)
        remaining_energy_needed = energy_needed - boost_needed
        total_available = self.battery.charge_level + self.supercapacitor.charge_level

        if total_available >= remaining_energy_needed:
            battery_ratio = self.battery.charge_level / total_available
            battery_usage = remaining_energy_needed * battery_ratio

            self.battery.discharge(battery_usage)
            self.total_energy_consumed += remaining_energy_needed
            self.total_distance_covered += distance
            self.terrain_energy_usage[terrain] += remaining_energy_needed

            wh_per_km = self.total_energy_consumed / self.total_distance_covered
            print(f"Energy Efficiency: {wh_per_km:.2f} Wh/km | Terrain: {terrain}")
        else:
            print("Warning: Not enough energy to complete trip!")

        # Optionally trigger regenerative braking if going downhill and speed is given
        if terrain == "Downhill" and speed is not None:
            self.regenerative_braking(speed, terrain)

    def regenerative_braking(self, speed, terrain):
        if terrain == "Downhill":
            # Use motor's power_rating if available, otherwise use a default value
            motor_power = getattr(self.motor, 'power_rating', 5000)
            recovered_energy = (motor_power * speed * 0.05)
            self.battery.charge(recovered_energy)
            print(f"Regenerative braking recovered {recovered_energy:.2f} Wh")

    def charge_solar(self, duration_hours):
        total_charge = 0
        for hour in range(duration_hours):
            sunlight_intensity = self.weather.get_sunlight(hour + 6)
            for minute in range(60):
                solar_input = (self.solar_panel.generate_power(sunlight_intensity) / 60)
                self.battery.charge(solar_input)
                total_charge += solar_input
            self.hourly_data.append((hour + 6, self.battery.charge_level))
        print(f"Battery trickle charged by {total_charge:.2f} Wh over {duration_hours} hours")


In [10]:
def regenerative_braking(self, speed, terrain):
    if terrain == "Downhill":
        recovered_energy = (self.motor_power * speed * 0.05)
        self.battery.charge(recovered_energy)
        print(f"Regenerative braking recovered {recovered_energy:.2f} Wh")


In [11]:
def charge_solar(self, duration_hours):
    total_charge = 0
    for hour in range(duration_hours):
        sunlight_intensity = self.weather.get_sunlight(hour + 6)
        for minute in range(60):
            solar_input = (self.solar_panel.generate_power(sunlight_intensity) / 60)
            self.battery.charge(solar_input)
            total_charge += solar_input
        self.hourly_data.append((hour + 6, self.battery.charge_level))

    print(f"Battery trickle charged by {total_charge:.2f} Wh over {duration_hours} hours")


In [12]:
def drive(self, distance, terrain):
    terrain_factor = {"Flat": 1.0, "Hill": 1.5, "Sandy": 1.8, "Rough": 2.0, "Downhill": 0.7}.get(terrain, 1.0)
    energy_needed = self.motor.energy_required(distance, terrain_factor)
    boost_needed = min(energy_needed * 0.2, self.supercapacitor.charge_level)

    self.supercapacitor.provide_boost(boost_needed)
    remaining_energy_needed = energy_needed - boost_needed
    total_available = self.battery.charge_level + self.supercapacitor.charge_level

    if total_available >= remaining_energy_needed:
        battery_ratio = self.battery.charge_level / total_available
        battery_usage = remaining_energy_needed * battery_ratio

        self.battery.discharge(battery_usage)
        self.total_energy_consumed += remaining_energy_needed
        self.total_distance_covered += distance
        self.terrain_energy_usage[terrain] += remaining_energy_needed

        wh_per_km = self.total_energy_consumed / self.total_distance_covered
        print(f"Energy Efficiency: {wh_per_km:.2f} Wh/km | Terrain: {terrain}")
    else:
        print("Warning: Not enough energy to complete trip!")


In [13]:
# Define terrains, distances, and speeds (add speed for downhill)
terrains = ["Flat", "Hill", "Sandy", "Rough", "Downhill"]
distances = [10, 10, 10, 10, 10]  # Example distances per terrain (km)
speeds = [30, 25, 20, 15, 40]  # Example speeds per terrain (km/h)

# Create the tuktuk
motor = Motor(power_rating=5000, efficiency=0.85)
solar_tuktuk = SolarTukTuk(
    battery_capacity=5000,
    capacitor_capacity=500,
    motor=motor,
    kerb_weight=400,
    top_speed=80,
    panel_area=1.5,
    panel_efficiency=0.2
)

# Solar charge before trips
solar_tuktuk.charge_solar(duration_hours=6)

# Simulate trips and collect metrics
results = []
for terrain, distance, speed in zip(terrains, distances, speeds):
    before_energy = solar_tuktuk.battery.charge_level + solar_tuktuk.supercapacitor.charge_level

    # Track recovered energy for this trip
    recovered_energy = 0
    # Drive as usual
    solar_tuktuk.drive(distance=distance, terrain=terrain, speed=speed if terrain == "Downhill" else None)
    # If downhill, call regenerative braking and track recovered energy
    if terrain == "Downhill":
        # Regenerative braking is now called inside drive, but if not, call it here:
        # solar_tuktuk.regenerative_braking(speed, terrain)
        # Calculate recovered energy (assuming method adds to battery directly)
        # For tracking, you can re-calculate or store in the class as an attribute
        # Here, let's recalculate for reporting:
        recovered_energy = (motor.power_rating * speed * 0.05)

    after_energy = solar_tuktuk.battery.charge_level + solar_tuktuk.supercapacitor.charge_level
    energy_used = before_energy - after_energy - recovered_energy  # Subtract recovered energy to get net use

    results.append({
        "terrain": terrain,
        "distance_km": distance,
        "energy_used_Wh": energy_used,
        "recovered_Wh": recovered_energy,
        "net_energy_Wh": energy_used - recovered_energy,
        "efficiency_Wh_per_km": (energy_used - recovered_energy) / distance if distance > 0 else None
    })

# Convert to DataFrame for easier analysis
import pandas as pd
df = pd.DataFrame(results)

# Show the DataFrame
print(df)


Battery trickle charged by 816.36 Wh over 6 hours
Energy Efficiency: 4.71 Wh/km | Terrain: Flat
Energy Efficiency: 5.88 Wh/km | Terrain: Hill
Energy Efficiency: 6.75 Wh/km | Terrain: Sandy
Energy Efficiency: 7.41 Wh/km | Terrain: Rough
Energy Efficiency: 6.59 Wh/km | Terrain: Downhill
Regenerative braking recovered 10000.00 Wh
    terrain  distance_km  energy_used_Wh  recovered_Wh  net_energy_Wh  \
0      Flat           10       55.669563           0.0      55.669563   
1      Hill           10       83.777596           0.0      83.777596   
2     Sandy           10      100.922677           0.0     100.922677   
3     Rough           10      112.634544           0.0     112.634544   
4  Downhill           10   -11954.291439       10000.0  -21954.291439   

   efficiency_Wh_per_km  
0              5.566956  
1              8.377760  
2             10.092268  
3             11.263454  
4          -2195.429144  


In [14]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

def run_simulation(
    battery_capacity=5000, 
    capacitor_capacity=500, 
    motor_power=5000, 
    motor_efficiency=0.85, 
    panel_area=1.5, 
    panel_efficiency=0.2,
    trickle_charge=True,
    solar_hours=6,
    distance_per_terrain=10,
    grid_cost_per_kwh=0.20,
    initial_investment_solar=9000,
    initial_investment_electric=6000,
    annual_km=20000,
    base_kerb_weight=400,
    panel_weight_per_m2=12.5,
    cap_weight_per_Wh=0.04
):
    terrains = ["Flat", "Hill", "Sandy", "Rough", "Downhill"]
    speeds = [30, 25, 20, 15, 40]

    # Calculate kerb weights
    solar_kerb_weight = base_kerb_weight + (panel_area * panel_weight_per_m2) + (capacitor_capacity * cap_weight_per_Wh)
    electric_kerb_weight = base_kerb_weight

    motor = Motor(power_rating=motor_power, efficiency=motor_efficiency)
    solar_tuktuk = SolarTukTuk(
        battery_capacity=battery_capacity,
        capacitor_capacity=capacitor_capacity,
        motor=motor,
        kerb_weight=solar_kerb_weight,
        top_speed=80,
        panel_area=panel_area,
        panel_efficiency=panel_efficiency
    )
    electric_tuktuk = ElectricTukTuk(
        battery_capacity=battery_capacity,
        motor=motor,
        kerb_weight=electric_kerb_weight,
        top_speed=80
    )
    if trickle_charge:
        print("[SolarTukTuk] ", end="")
        solar_tuktuk.charge_solar(duration_hours=solar_hours)
    else:
        print("[SolarTukTuk] Skipping trickle charging.")

    results = []
    for terrain, speed in zip(terrains, speeds):
        distance = distance_per_terrain

        # Solar
        before_energy_solar = solar_tuktuk.battery.charge_level + solar_tuktuk.supercapacitor.charge_level
        energy_needed_solar = motor.energy_required(
            distance, 
            {"Flat": 1.0, "Hill": 1.5, "Sandy": 1.8, "Rough": 2.0, "Downhill": 0.7}[terrain],
            kerb_weight=solar_kerb_weight
        )
        recovered_solar = min(energy_needed_solar * 0.1, motor.power_rating * speed * 0.005) if terrain == "Downhill" else 0
        solar_tuktuk.drive(distance=distance, terrain=terrain, speed=speed if terrain == "Downhill" else None)
        after_energy_solar = solar_tuktuk.battery.charge_level + solar_tuktuk.supercapacitor.charge_level
        net_energy_used_solar = before_energy_solar - after_energy_solar - recovered_solar

        # Electric
        before_energy_electric = electric_tuktuk.battery.charge_level
        energy_needed_electric = motor.energy_required(
            distance, 
            {"Flat": 1.0, "Hill": 1.5, "Sandy": 1.8, "Rough": 2.0, "Downhill": 0.7}[terrain],
            kerb_weight=electric_kerb_weight
        )
        recovered_electric = min(energy_needed_electric * 0.1, motor.power_rating * speed * 0.005) if terrain == "Downhill" else 0
        electric_tuktuk.drive(distance=distance, terrain=terrain, speed=speed if terrain == "Downhill" else None)
        after_energy_electric = electric_tuktuk.battery.charge_level
        net_energy_used_electric = before_energy_electric - after_energy_electric - recovered_electric

        # Range calculation (fresh vehicles to avoid depletion)
        def calc_range(vehicle, terrain, speed, kerb_weight, trickle_charge, solar_hours=6):
            if trickle_charge and hasattr(vehicle, 'charge_solar'):
                vehicle.charge_solar(duration_hours=solar_hours)
            terrain_factor = {"Flat": 1.0, "Hill": 1.5, "Sandy": 1.8, "Rough": 2.0, "Downhill": 0.7}[terrain]
            energy_per_km = vehicle.motor.energy_required(1, terrain_factor, kerb_weight=kerb_weight)
            if terrain == "Downhill":
                recovered_per_km = min(energy_per_km * 0.1, vehicle.motor.power_rating * speed * 0.005 / 1)
            else:
                recovered_per_km = 0
            net_energy_per_km = max(energy_per_km - recovered_per_km, 0.01)
            if hasattr(vehicle, 'supercapacitor'):
                available_energy = vehicle.battery.charge_level + vehicle.supercapacitor.charge_level
            else:
                available_energy = vehicle.battery.charge_level
            return available_energy / net_energy_per_km

        solar_vehicle = SolarTukTuk(
            battery_capacity=battery_capacity,
            capacitor_capacity=capacitor_capacity,
            motor=motor,
            kerb_weight=solar_kerb_weight,
            top_speed=80,
            panel_area=panel_area,
            panel_efficiency=panel_efficiency
        )
        electric_vehicle = ElectricTukTuk(
            battery_capacity=battery_capacity,
            motor=motor,
            kerb_weight=electric_kerb_weight,
            top_speed=80
        )
        solar_range = calc_range(solar_vehicle, terrain, speed, solar_kerb_weight, trickle_charge, solar_hours)
        electric_range = calc_range(electric_vehicle, terrain, speed, electric_kerb_weight, False, 0)

        results.append({
            "terrain": terrain,
            "distance_km": distance,
            "solar_eff_Wh_per_km": net_energy_used_solar / distance if distance > 0 else None,
            "electric_eff_Wh_per_km": net_energy_used_electric / distance if distance > 0 else None,
            "solar_range_km": solar_range,
            "electric_range_km": electric_range
        })

    df = pd.DataFrame(results)
    # Calculate running cost per km
    df['solar_cost_per_km'] = df['solar_eff_Wh_per_km'] / 1000 * 0  # Solar running cost (USD)
    df['electric_cost_per_km'] = df['electric_eff_Wh_per_km'] / 1000 * grid_cost_per_kwh

    # Annual running cost (per terrain)
    df['solar_annual_cost'] = df['solar_cost_per_km'] * annual_km
    df['electric_annual_cost'] = df['electric_cost_per_km'] * annual_km

    # Total cost over one year (initial + running)
    df['solar_total_cost_year'] = initial_investment_solar + df['solar_annual_cost']
    df['electric_total_cost_year'] = initial_investment_electric + df['electric_annual_cost']

    # Plot all metrics
    fig, axs = plt.subplots(1, 3, figsize=(22,6))

    # 1. Energy Efficiency Plot
    axs[0].bar(df['terrain'], df['solar_eff_Wh_per_km'], width=0.4, label='SolarTukTuk', align='center')
    axs[0].bar(df['terrain'], df['electric_eff_Wh_per_km'], width=0.4, label='ElectricTukTuk', align='edge')
    axs[0].set_xlabel("Terrain")
    axs[0].set_ylabel("Net Energy Efficiency (Wh/km)")
    axs[0].set_title(f"Energy Efficiency by Terrain\n(Distance: {distance_per_terrain} km)")
    axs[0].legend()
    axs[0].grid(True)
    axs[0].set_ylim(0, max(df['solar_eff_Wh_per_km'].max(), df['electric_eff_Wh_per_km'].max()) * 1.2)

    # 2. Range Plot
    bar_width = 0.35
    x = range(len(df['terrain']))
    axs[1].bar([i - bar_width/2 for i in x], df['solar_range_km'], width=bar_width, label='SolarTukTuk')
    axs[1].bar([i + bar_width/2 for i in x], df['electric_range_km'], width=bar_width, label='ElectricTukTuk')
    axs[1].set_xticks(x)
    axs[1].set_xticklabels(df['terrain'])
    axs[1].set_xlabel("Terrain")
    axs[1].set_ylabel("Estimated Range (km)")
    axs[1].set_title(f"Estimated Range by Terrain\n(Trickle Charging: {'ON' if trickle_charge else 'OFF'})")
    axs[1].legend()
    axs[1].grid(True)
    axs[1].set_ylim(0, max(df['solar_range_km'].max(), df['electric_range_km'].max()) * 1.2)

    # 3. Total Cost Over 1 Year Plot
    axs[2].bar([i - bar_width/2 for i in x], df['solar_total_cost_year'], width=bar_width, label='SolarTukTuk')
    axs[2].bar([i + bar_width/2 for i in x], df['electric_total_cost_year'], width=bar_width, label='ElectricTukTuk')
    axs[2].set_xticks(x)
    axs[2].set_xticklabels(df['terrain'])
    axs[2].set_xlabel("Terrain")
    axs[2].set_ylabel("Total Cost (USD, 1st year)")
    axs[2].set_title(f"Total Cost (Initial + Running, 1 Year, {annual_km} km)")
    axs[2].legend()
    axs[2].grid(True)
    axs[2].set_ylim(0, max(df['solar_total_cost_year'].max(), df['electric_total_cost_year'].max()) * 1.2)

    plt.tight_layout()
    plt.show()

    # --- Energy Efficiency DataFrame ---
    df_efficiency = df[['terrain', 'solar_eff_Wh_per_km', 'electric_eff_Wh_per_km']]
    df_efficiency.columns = ['Terrain', 'SolarTukTuk (Wh/km)', 'ElectricTukTuk (Wh/km)']
    print("Energy Efficiency (Wh/km):")
    display(df_efficiency)

    # --- Range DataFrame ---
    df_range = df[['terrain', 'solar_range_km', 'electric_range_km']]
    df_range.columns = ['Terrain', 'SolarTukTuk Range (km)', 'ElectricTukTuk Range (km)']
    print("Estimated Range (km):")
    display(df_range)

    # --- Running Cost DataFrame ---
    df_cost = df[['terrain', 'solar_cost_per_km', 'electric_cost_per_km', 'solar_total_cost_year', 'electric_total_cost_year']]
    df_cost.columns = [
        'Terrain', 
        'SolarTukTuk Cost/km (USD)', 
        'ElectricTukTuk Cost/km (USD)', 
        'SolarTukTuk Total 1st Year Cost (USD)', 
        'ElectricTukTuk Total 1st Year Cost (USD)'
    ]
    print("Running Cost:")
    display(df_cost)

# UI controls
interact(
    run_simulation,
    battery_capacity=widgets.IntSlider(min=2000, max=10000, step=500, value=5000, description='Battery (Wh)'),
    capacitor_capacity=widgets.IntSlider(min=0, max=2000, step=100, value=500, description='Capacitor (Wh)'),
    motor_power=widgets.IntSlider(min=1000, max=10000, step=500, value=5000, description='Motor Power (W)'),
    motor_efficiency=widgets.FloatSlider(min=0.7, max=0.98, step=0.01, value=0.85, description='Motor Eff.'),
    panel_area=widgets.FloatSlider(min=0.5, max=3.0, step=0.1, value=1.5, description='Panel Area (m²)'),
    panel_efficiency=widgets.FloatSlider(min=0.1, max=0.25, step=0.01, value=0.2, description='Panel Eff.'),
    trickle_charge=widgets.Checkbox(value=True, description="Trickle Charge (Solar)"),
    solar_hours=widgets.IntSlider(min=0, max=12, step=1, value=6, description='Solar Hours'),
    distance_per_terrain=widgets.IntSlider(min=1, max=50, step=1, value=10, description='Distance (km)'),
    grid_cost_per_kwh=widgets.FloatSlider(min=0.05, max=0.50, step=0.01, value=0.20, description='Grid Cost ($/kWh)'),
    initial_investment_solar=widgets.IntSlider(min=5000, max=20000, step=500, value=9000, description='Solar Initial ($)'),
    initial_investment_electric=widgets.IntSlider(min=3000, max=15000, step=500, value=6000, description='Electric Initial ($)'),
    annual_km=widgets.IntSlider(min=5000, max=50000, step=1000, value=20000, description='Annual km'),
    base_kerb_weight=widgets.IntSlider(min=300, max=700, step=10, value=400, description='Base Kerb Wt (kg)'),
    panel_weight_per_m2=widgets.FloatSlider(min=8, max=20, step=0.5, value=12.5, description='Panel Wt (kg/m²)'),
    cap_weight_per_Wh=widgets.FloatSlider(min=0.02, max=0.08, step=0.005, value=0.04, description='Cap Wt (kg/Wh)')
)


interactive(children=(IntSlider(value=5000, description='Battery (Wh)', max=10000, min=2000, step=500), IntSli…

<function __main__.run_simulation(battery_capacity=5000, capacitor_capacity=500, motor_power=5000, motor_efficiency=0.85, panel_area=1.5, panel_efficiency=0.2, trickle_charge=True, solar_hours=6, distance_per_terrain=10, grid_cost_per_kwh=0.2, initial_investment_solar=9000, initial_investment_electric=6000, annual_km=20000, base_kerb_weight=400, panel_weight_per_m2=12.5, cap_weight_per_Wh=0.04)>

In [15]:
import streamlit as st
import matplotlib.pyplot as plt
import pandas as pd
from simulation import run_simulation  # Make sure this is your simulation function

st.title("Tuktuk Solar vs Electric Simulation Dashboard")

# Sidebar for user input
battery_capacity = st.sidebar.slider("Battery Capacity (Wh)", 2000, 10000, 5000, 500)
capacitor_capacity = st.sidebar.slider("Capacitor Capacity (Wh)", 0, 2000, 500, 100)
panel_area = st.sidebar.slider("Panel Area (m²)", 0.5, 3.0, 1.5, 0.1)
# Add more sliders for all your simulation parameters as needed

if st.button("Run Simulation"):
    # The run_simulation function should return a dictionary like:
    # {
    #   'efficiency_plot': matplotlib.figure.Figure,
    #   'range_plot': matplotlib.figure.Figure,
    #   'cost_plot': matplotlib.figure.Figure,
    #   'df_efficiency': pd.DataFrame,
    #   'df_range': pd.DataFrame,
    #   'df_cost': pd.DataFrame
    # }
    results = run_simulation(
        battery_capacity=battery_capacity,
        capacitor_capacity=capacitor_capacity,
        panel_area=panel_area,
        # ...all other parameters...
    )
    # Show plots
    st.subheader("Energy Efficiency by Terrain")
    st.pyplot(results['efficiency_plot'])
    st.subheader("Estimated Range by Terrain")
    st.pyplot(results['range_plot'])
    st.subheader("Total Cost Over 1 Year")
    st.pyplot(results['cost_plot'])
    # Show DataFrames
    st.subheader("Energy Efficiency Table")
    st.dataframe(results['df_efficiency'])
    st.subheader("Range Table")
    st.dataframe(results['df_range'])
    st.subheader("Running Cost Table")
    st.dataframe(results['df_cost'])


Battery trickle charged by 929.76 Wh over 6 hours
Energy Efficiency: 4.71 Wh/km | Terrain: Flat
Energy Efficiency: 5.88 Wh/km | Terrain: Hill
Energy Efficiency: 6.75 Wh/km | Terrain: Sandy
Energy Efficiency: 7.41 Wh/km | Terrain: Rough
Energy Efficiency: 6.59 Wh/km | Terrain: Downhill
Regenerative braking recovered 10000.00 Wh
    terrain  distance_km  energy_used_Wh  recovered_Wh  net_energy_Wh  \
0      Flat           10       55.767072           0.0      55.767072   
1      Hill           10       83.917646           0.0      83.917646   
2     Sandy           10      101.082172           0.0     101.082172   
3     Rough           10      112.800387           0.0     112.800387   
4  Downhill           10   -11841.454335       10000.0  -21841.454335   

   efficiency_Wh_per_km  
0              5.576707  
1              8.391765  
2             10.108217  
3             11.280039  
4          -2184.145433  


interactive(children=(IntSlider(value=5000, description='battery_capacity', max=10000, min=1000, step=100), In…

2025-07-11 22:15:36.823 
  command:

    streamlit run /home/mugogoinc/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
